In [31]:
from glossary.Glossary import Glossary
import uuid
from glossary.glossaries.pretty_glossary import pretty_core
glossary = Glossary(pretty_core)
import pymongo
import spacy

In [32]:
from bs4 import BeautifulSoup
import collections
import urllib
import requests
import nltk
from nltk.corpus import stopwords
import textacy
from urllib.request import Request, urlopen
import textacy.keyterms as tkt

In [33]:
def create_connection():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["parent_child1"]
    mycol = mydb["tags"]
    return mycol

In [34]:
#create 'index'
#input : list of dict, key for index
#output : dict with value = dict from input, key = key for index
def build_dict(seq, key):
    return dict((d[key], dict(d, index=index)) for (index, d) in enumerate(seq))

In [35]:
def create_str_name_id(sort_glos):
    name_id=[]
    for i in sort_glos:
        doc = {'_id' : str(uuid.uuid4()), 'name' : i}
        name_id.append(doc)
    return name_id

In [36]:
def create_doc_for_mongo(sort_glos,final):
    for i in sort_glos:
        p=glossary.get_parent(i)
        c=glossary.get_children(i)
        ch = [val for sublist in c for val in sublist]
        index = []
        id1 = []
        id2=[]
        for  j in range(len(p)):
            index.append(next((index for (index, d) in enumerate(name_id) if d["name"] == p[j]), None))
            if any(elem is not None for elem in index) : id1.append(name_id[index[j]].get('_id'))

        for  k in range(len(ch)):
            child_info = info_by_name.get(ch[k])
            id2.append(child_info["_id"])
        par = {'parent':id1}
        child = {'child':id2}
        doc = {'_id' : info_by_name.get(i)['_id'], 'name' : i, 'parent' : id1,'child' : id2}
        final.append(doc)
        return final

In [37]:
#insert into DB
def insert(mycol,final):
    x = mycol.insert_many(final)

In [38]:
def matching(url='https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8'):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
    except:
        print("Smth wrong")
    soup = BeautifulSoup( webpage,'html.parser')
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    #findng key phrases
    text = textacy.preprocess_text(text, lowercase=True, no_punct=True,no_urls=True)
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    text = ' '.join(filtered_sentence)
    doc = textacy.Doc(text,lang='en_core_web_md')
    our_list = list(tkt.sgrank(doc, ngrams=(1, 2, 3), normalize='lower', n_keyterms=0.1, window_width=1500))[:20]
    our_list_withot_probs = [i for i,j in our_list]
    return our_list,our_list_withot_probs

In [39]:
def get_category(child, category_for_word):
    if str(child.get('name')) not in category :
        parent_id = child.get('parent')
        #print(parent_id)
        for i in parent_id:
            #print(i)
            parent = mycol.find({'_id':str(i)},{'name':1,'parent':1})
            #print(parent)
            for j in parent:
                #print(j)
                #if str(j.get('name')) not in category:
                 #   print(j)
                get_category(j,category_for_word)
                #else : category_for_word.append(j.get(name))
    else : 
        category_for_word.append(child.get('name') )
        return category_for_word 

In [44]:
def match_category(category_for_word):
    for i in mb_category:
        for j in mycol.find({},{'_id':1,'name':1,'parent':1}):
            if str(i)==str(j.get('name')):
                get_category(j,category_for_word)
            else: 
                s = textacy.similarity.hamming(str(i), str(j.get('name')) )
                if s>threshold:

                    get_category(j,category_for_word)


In [46]:
mycol = create_connection()
sort_glos = sorted(glossary.tags)
name_id = create_str_name_id(sort_glos)
info_by_name=build_dict(name_id,key = 'name')
final =[]
create_doc_for_mongo(sort_glos,final)
insert(mycol,final)
mb_category=matching()[1]
category = ['artificial intelligence','deep learning','computer vision','natural language processing']
threshold = 0.4
category_for_word =[]
category_in_text = match_category(category_for_word)
